In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
import os
print("Current working directory:", os.getcwd())
print("\nFiles in current directory:")
print(os.listdir('.'))

In [ ]:
data=pd.read_csv('Churn_Modelling.csv')

In [ ]:
data.head()

In [ ]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [ ]:
data

In [ ]:
le=LabelEncoder()
data['Gender']=le.fit_transform(data['Gender'])

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe=OneHotEncoder(sparse_output=False)
geo_encoder=ohe.fit_transform(data[['Geography']])
geo_encoder

In [ ]:
geo_encoder_df=pd.DataFrame(geo_encoder,columns=ohe.get_feature_names_out())

In [ ]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)
data.head()

In [ ]:
encoders = {
    'label_encoder_gender': le,
    'one_hot_encoder_geography': ohe,
}
with open('encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
x=data.drop('Exited',axis=1)
y=data['Exited']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
x_train

In [ ]:
# Update the existing encoders dict with the scaler and rewrite the pickle file
encoders['scaler'] = scaler
with open('encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f, protocol=pickle.HIGHEST_PROTOCOL)

ANN implementation

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorboard

In [19]:
(x_train.shape[1],)

(12,)

In [20]:
model=Sequential([
    Dense(64,activation='relu',input_shape=((x_train.shape[1],))),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

c:\Users\theni\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [23]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [24]:
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(f"Log directory: {log_dir}")

Log directory: logs/fit/20251218-161513


In [25]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
tensorflow_callback=TensorBoard(log_dir,histogram_freq=1)

In [26]:
early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [27]:
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8147 - loss: 0.4371 - val_accuracy: 0.8515 - val_loss: 0.3651
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8555 - loss: 0.3542 - val_accuracy: 0.8615 - val_loss: 0.3429
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8556 - loss: 0.3466 - val_accuracy: 0.8640 - val_loss: 0.3465
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8601 - loss: 0.3393 - val_accuracy: 0.8530 - val_loss: 0.3460
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8616 - loss: 0.3457 - val_accuracy: 0.8600 - val_loss: 0.3424
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8624 - loss: 0.3347 - val_accuracy: 0.8635 - val_loss: 0.3452
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8596 - loss: 0.3389 - val_accuracy: 0.8600 - val_loss: 0.3425
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8710 - loss: 0.3226 - val_accu

In [28]:
model.save('my_model.h5')

In [29]:
%load_ext tensorboard

In [30]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 17676), started 18:29:28 ago. (Use '!kill 17676' to kill it.)

In [31]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model